# Graph Neural Networks

Graph Neural Networks (GNNs) are a class of neural networks that operate on graphs. They are a powerful tool for solving problems in domains such as social network analysis, recommender systems, and combinatorial optimization.

You can read about Graph data structures /algorithms/graphs.html#graphs-data-structure section.

## Node Representations

The goal of a GNN is to learn a function that maps a graph to a representation of its nodes. This representation can then be used for various downstream tasks, such as node classification, link prediction, and clustering.

### DeepWalk

DeepWalk is a simple algorithm for learning node representations in a graph. It works by performing random walks on the graph, and using the SkipGram model from Word2Vec to learn the embeddings of the nodes.

It introduces important concepts such as embeddings that are at the core of GNNs. Unlike traditional neural networks, the goal of this architecture is to produce representations that are then fed to other models, which perform downstream tasks (for example, node classification).

DeepWalk architecture and its two major components: **Word2Vec** and **random walks**.

#### Word2Vec

The first step to comprehending the DeepWalk algorithm is to understand its major component: Word2Vec.
it proposed a new technique to translate words into vectors (also known as embeddings) using large datasets of text. These representations can then be used in downstream tasks, such as sentiment classification.

One of the most surprising results of Word2Vec is its ability to solve analogies. A popular example is how it can answer the question “man is to woman, what king is to ___?” It can be calculated as follows:

##### CBOW versus skip-gram
its only goal is to produce high-quality embeddings.

The continuous bag-of-words (CBOW) model:

This is trained to predict a word using its surrounding context (words coming before and after the target word). The order of context words does not matter since their embeddings are summed in the model. The authors claim to obtain better results using four words before and after the one that is predicted.

The continuous skip-gram model:

Here, we feed a single word to the model and try to predict the words around it. Increasing the range of context words leads to better embeddings but also increases the training time.

```{image} https://i.stack.imgur.com/ShJJX.png
:alt: CBOW and skip-gram models
:width: 80%
:align: center
```

##### Creating skip-grams

For now, we will focus on the skip-gram model since it is the architecture used by DeepWalk. Skip-grams are implemented as pairs of words with the following structure ( target word, context word). where target word is the input and context word is the word to predict. The number of skip grams for the same target word depends on a parameter called context size.


| Context Size | Text | Skip-grams |
| :---: | :---: | :---: |
| 1 | the train was late. | ('the', 'train') |
|  | the train was late | $\begin{array}{l}\text { ('train', 'the') } \\\text { ('train', 'was') }\end{array}$ |
|  | the train was late | $\begin{array}{l}\text { ('was', 'train') } \\\text { ('was', 'late') }\end{array}$ |
|  | the train was late | ('late', 'was') |
| 2 | the train was late | $\begin{array}{l}\text { ('the', 'train') } \\\text { ('the', 'was') }\end{array}$ |
|  | the train was late | $\begin{array}{l}\text { ('train', 'the') } \\\text { ('train', 'was') } \\\text { ('train', 'late') }\end{array}$ |
|  | the train was late | $\begin{array}{l}\text { ('was', 'the') } \\\text { ('was', 'train') } \\\text { ('was', 'late') }\end{array}$ |
|  | the train was late | $\begin{array}{l}\text { ('late', 'train') } \\\text { ('late', 'was') }\end{array}$ |



